# **ENGG680 - Introduction to Digital Engineering**
## *Lab Assignment 4: ML Applications - Vehicle Motion State Classification*

## Preliminary: Certificate of Work


*We, the undersigned, certify that this is our own work, which has been done expressly for this course, either without the assistance of any other party or where appropriate we have acknowledged the work of others. Further, we have read and understood the section in the university calendar on plagiarism/cheating/other academic misconduct and we are aware of the implications thereof. We request that the total mark for this assignment be distributed as follows among group members:*

|          | First Name | Last Name | Signature (Full Name, Date) | Hours | Contribution % |
|----------|------------|-----------|-----------------------------|-------|----------------|
| Member 1: | Sahar | Yousefi | Sahar Yousefi, 11/29/2024 | 10 Hours | 25% |
| Member 2: | Wedeb | Teweldemedhin | Wedeb Teweldemedhin, 11/29/2024 | 10 Hours | 25% |
| Member 3: | Bahram | Akbari | Bahram Akbari, 11/29/2024 | 10 Hours | 25% |
| Member 4: | Nishesh | Nepal | 	Nishesh Nepal, 11/29/2024 | 10 Hours | 25% |


---

## Section 1 - Motion Classification Using *All Features* Derived From Inertial Sensor Data (33 Marks)

---

### 1.1 Assignment datasets import (6 Marks)

In [9]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---

In [10]:
# some library imports
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier

First load the following datasets into dataframes.
* *UrbanNav-HK_Whampoa-20210521_sensors.csv* to dataframe variable named *cv_set*. This dataset is to be used for model training and hyper parameter tuning.
* *UrbanNav-HK_TST-20210517_sensors.csv* to dataframe variable named *test_set1*. This is first independent test dataset.
* *2020-03-14-16-45-35.csv* to dataframe variable named *test_set2*. This is the second independent test dataset.

In [11]:
# (2 marks)
# TODO: Load cross validation/training dataset (UrbanNav-HK_Whampoa-20210521_sensors.csv) to 'cv_set' dataframe

# uncomment and compelete below
cv_set = pd.read_csv('/content/drive/MyDrive/engg680_2024_fall/Lab-Assignment4/UrbanNav-HK_Whampoa-20210521_sensors.csv')

# TODO: Load UrbanNav-HK_TST-20210517_sensors.csv to 'test_set1' dataframe
test_set1 = pd.read_csv('/content/drive/MyDrive/engg680_2024_fall/Lab-Assignment4/UrbanNav-HK_TST-20210517_sensors.csv')

# TODO: Load 2020-03-14-16-45-35.csv to 'test_set2' dataframe
test_set2 = pd.read_csv('/content/drive/MyDrive/engg680_2024_fall/Lab-Assignment4/2020-03-14-16-45-35.csv')

For the *cv_set*, look at the dataframe head and info, and print the list of all columns. The *test_set1* and *test_set2* have the same schema as *cv_set*, however, have different number of rows (samples).

In [12]:
# TODO: Display first two rows of cv_set
# (1 mark)
display(cv_set.head(2))

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max,motion_state
0,-0.003929,-0.093081,9.789349,-0.002649,0.001406,-0.001391,9.797418,0.022554,8.046094e-07,1.319665e-07,...,0.000003,0.001496,0.000544,0.024992,0.000110,0.000075,0.000078,0.024997,0.000187,rest
1,-0.000966,-0.092417,9.789095,-0.001806,0.000103,-0.001943,9.798815,0.021731,1.032619e-06,1.073067e-07,...,0.000006,0.001865,0.000615,0.024828,0.000124,0.000098,0.000090,0.024829,0.000144,rest


In [13]:
# TODO: Load the info method of cv_set
# (1 mark)
cv_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534 entries, 0 to 1533
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ax_sum        1534 non-null   float64
 1   ay_sum        1534 non-null   float64
 2   az_sum        1534 non-null   float64
 3   gx_sum        1534 non-null   float64
 4   gy_sum        1534 non-null   float64
 5   gz_sum        1534 non-null   float64
 6   a_sum         1534 non-null   float64
 7   g_sum         1534 non-null   float64
 8   ax_var        1534 non-null   float64
 9   ay_var        1534 non-null   float64
 10  az_var        1534 non-null   float64
 11  gx_var        1534 non-null   float64
 12  gy_var        1534 non-null   float64
 13  gz_var        1534 non-null   float64
 14  a_var         1534 non-null   float64
 15  g_var         1534 non-null   float64
 16  ax_min        1534 non-null   float64
 17  ay_min        1534 non-null   float64
 18  az_min        1534 non-null 

In [14]:
# TODO: Display the number of rows and columns  of all three datasets
# (1 mark)
#print(f"'cv_set' has {???} rows and {???} columns")
#print(f"'test_set1' has {???} rows and {???} columns")
#print(f"'test_set2' has {???} rows and {???} columns")

print(f"'cv_set' has {cv_set.shape[0]} rows and {cv_set.shape[1]} columns")
print(f"'test_set1' has {test_set1.shape[0]} rows and {test_set1.shape[1]} columns")
print(f"'test_set2' has {test_set2.shape[0]} rows and {test_set2.shape[1]} columns")

'cv_set' has 1534 rows and 33 columns
'test_set1' has 783 rows and 33 columns
'test_set2' has 297 rows and 33 columns


In [15]:
# TODO: Display all columns in cv_set
# (1 mark)
display(cv_set.columns)

Index(['ax_sum', 'ay_sum', 'az_sum', 'gx_sum', 'gy_sum', 'gz_sum', 'a_sum',
       'g_sum', 'ax_var', 'ay_var', 'az_var', 'gx_var', 'gy_var', 'gz_var',
       'a_var', 'g_var', 'ax_min', 'ay_min', 'az_min', 'gx_min', 'gy_min',
       'gz_min', 'a_min', 'g_min', 'ax_max', 'ay_max', 'az_max', 'gx_max',
       'gy_max', 'gz_max', 'a_max', 'g_max', 'motion_state'],
      dtype='object')

In all three dataframes, target vector is stored in the *motion_state* column. All other columns are features.

___

### 1.2 - Grid Search with Cross Validation (18 Marks)

___

#### 1.2.1 - Extract all features and target from cv_set

___

From the *cv_set*, load the target vector stored in the *motion_state* column into a new variable *y*

In [16]:
# TODO - Load target vector into y
# (1 mark)
# y =
y = cv_set["motion_state"]

From *cv_set*, load all features, that is all columns except for the target vector, in a new dataframe variable *X*

In [17]:
# TODO - Load all features from cv_set to X
# (2 marks)
# X =
X = cv_set.drop('motion_state', axis=1)
X.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.003929,-0.093081,9.789349,-0.002649,0.001406,-0.001391,9.797418,0.022554,8.046094e-07,1.319665e-07,...,0.023871,0.000003,0.001496,0.000544,0.024992,0.000110,0.000075,0.000078,0.024997,0.000187
1,-0.000966,-0.092417,9.789095,-0.001806,0.000103,-0.001943,9.798815,0.021731,1.032619e-06,1.073067e-07,...,0.024153,0.000006,0.001865,0.000615,0.024828,0.000124,0.000098,0.000090,0.024829,0.000144


___

#### 1.2.2 - Configure and Perform Grid Search with Cross-Validation

___

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.model_selection import GridSearchCV

The dataset *(X, y)*, must be split into training/validation *(X_train, y_train)* and testing *(X_test, y_test)* sets. Use the `train_test_split` function from sklearn to create the split, where  **30%** of data is used for testing. Set the `random_state=42`, and `stratify=y`.  

In [19]:
# TODO - Perform test/train split
# (1 mark)
# X_train, X_test, y_train, y_test =
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

Initialize a `Pipeline`, consisting of a *scaler* and a *classifer*, set the default values to *StandardScaler()*, and *SVC()*, respectively.

In [20]:
# TODO - Initialize Pipeline into variable pipe
# (2 marks)
# pipe =
pipe = Pipeline([("scaler", StandardScaler()), ("svc", SVC())])

Define the grid search configuration (*param_grid*) for training and hyper parameter tuning of the *support vector classifier*. Tune, the hyper-parameter *C*, over the interval `C=[0.01, 0.1, 1 ,10,100]`, *gamma*, over the interval `gamma=[0.001, 0.01, 0.1,1]`, and use the *radial basis function* as the kernel. For the *scaler* parameter, test with the `StandardScaler`, `RobustScaler`, and `None` options.

In [21]:
# TODO - Setup grid set
# (4 marks)
# param_grid =

param_grid = {
    "scaler": [StandardScaler(), RobustScaler(), None],
    "svc__C": [0.01, 0.1, 1, 10, 100],
    "svc__gamma": [0.001, 0.01, 0.1, 1],
    "svc__kernel": ["rbf"],
}

In [22]:
# Perform the grid search to train model and tune hyper parameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring="accuracy", return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'scaler': [StandardScaler(), RobustScaler(), None],
                         'svc__C': [0.01, 0.1, 1, 10, 100],
                         'svc__gamma': [0.001, 0.01, 0.1, 1],
                         'svc__kernel': ['rbf']},
             return_train_score=True, scoring='accuracy')

___

#### 1.2.3 - Visualize Grid Search Results

___

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [23]:
# TODO: Print the results from the grid search
# (2 marks)

#print("Best params:\n{}\n".format(???))
#print("Best cross-validation train score: {:.2f}".format(???))
#print("Best cross-validation validation score: {:.2f}".format(???))
#print("Test-set score: {:.2f}".format(???)))

print("Task 1: SVC using all features")
print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format(grid.cv_results_['mean_train_score'][grid.best_index_]))
print("Best cross-validation validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Task 1: SVC using all features
Best params:
{'scaler': RobustScaler(), 'svc__C': 100, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}

Best cross-validation train score: 0.98
Best cross-validation validation score: 0.95
Test-set score: 0.96


Display the confusion matrix and print the classification report for the trained model. Use the *(X_test, y_test)* dataset. **Hint** - You can use your code from assignment 3.

In [24]:
# TODO - Display confusion matrix and print classification report for (X_test, y_test)
# (2 marks)
print("Confusion Matrix:\n", confusion_matrix(y_test, grid.predict(X_test)))
print("Classification Report:\n", classification_report(y_test, grid.predict(X_test)))

Confusion Matrix:
 [[153   2   0]
 [ 10 192   5]
 [  0   3  96]]
Classification Report:
               precision    recall  f1-score   support

        rest       0.94      0.99      0.96       155
    straight       0.97      0.93      0.95       207
        turn       0.95      0.97      0.96        99

    accuracy                           0.96       461
   macro avg       0.95      0.96      0.96       461
weighted avg       0.96      0.96      0.96       461



___

#### 1.2.4 - Questions

1. Comment on the fit of the model to the dataset? (2 marks)
2. In the grid search, are the intervals used for *C* and *gamma* hyperparameters suitable? Why or why not? (2 marks)

___

#### 1.2.4 - Answers
1.
The model fits the dataset well, achieving:
High cross-validation accuracy (95%) and test accuracy (96%), indicating strong generalization to unseen data.
A near-perfect training accuracy (100%), which could suggest slight overfitting.

2.
Yes, the intervals are suitable:
The C range [0.01, 0.1, 1, 10, 100] explores low to high regularization, balancing underfitting and overfitting.
The gamma range [0.001, 0.01, 0.1, 1] spans simple to complex decision boundaries, appropriate for the RBF kernel.
However, finer steps around the best-performing parameters (e.g., C=100 and gamma=0.01) could potentially further optimize the model.

### 1.3 - Model Evaluation on Test Datasets 1 and 2 (9 Marks)

___

#### 1.3.1 - Extract all features and targets from test datasets

Next, the model trained in 1.2 will be tested on independent datasets, that is *test_set1* and *test_set2*. <br>
The test_sets must be seperated into features (Xs) and targets (ys).

In [25]:
# TODO - Load the target vector from test_set1 into y1, and all features from test_set1 into X1
# (1 marks)
# y1 =
# X1 =
y1 = test_set1['motion_state']
X1 = test_set1.drop('motion_state', axis=1)
X1.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,0.264465,0.089904,9.786930,-0.002305,0.000218,-0.001143,9.801661,0.025634,0.000001,7.749919e-08,...,0.024056,0.000003,0.002506,0.000845,0.024884,0.000155,0.000078,0.000069,0.024886,0.000170
1,0.264300,0.088882,9.788127,-0.001865,-0.000523,-0.001999,9.802481,0.025218,0.000001,9.243000e-08,...,0.024084,0.000007,0.002427,0.000949,0.024881,0.000201,0.000095,0.000076,0.024883,0.000201


In [26]:
# TODO - Load the target vector from test_set2 into y2, and all features from test_set2 into X2
# y2 =
# X2 =
y2 = test_set2['motion_state']
X2 = test_set2.drop('motion_state', axis=1)
X2.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.046400,0.305907,9.791451,-0.002184,0.001311,-0.001901,9.800683,0.012437,0.000002,3.020959e-07,...,0.048406,0.000007,0.002575,0.002993,0.049701,0.000094,0.000115,0.000080,0.049764,0.000142
1,-0.050073,0.311947,9.785619,-0.003369,0.001898,-0.002051,9.797741,0.013805,0.000003,6.859937e-07,...,0.047696,0.000013,0.003880,0.004106,0.050477,0.000140,0.000140,0.000083,0.050763,0.000151


____

#### 1.3.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [27]:
# TODO - Print test score accuracies for test_set1 and test_set2
# (1 marks)

#print("test_set1 score: {:.2f}".format(???))
#print("test_set2 score: {:.2f}".format(???))

print("Test Set 1 Accuracy: {:.2f}".format(grid.score(X1, y1)))
print("Test Set 2 Accuracy: {:.2f}".format(grid.score(X2, y2)))

Test Set 1 Accuracy: 0.92
Test Set 2 Accuracy: 0.25


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [28]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
# (1 marks)

print("Confusion Matrix for Test Set 1:\n", confusion_matrix(y1, grid.predict(X1)))
print("Classification Report for Test Set 1:\n", classification_report(y1, grid.predict(X1)))

Confusion Matrix for Test Set 1:
 [[200  31   0]
 [  7 387  17]
 [  0   7 134]]
Classification Report for Test Set 1:
               precision    recall  f1-score   support

        rest       0.97      0.87      0.91       231
    straight       0.91      0.94      0.93       411
        turn       0.89      0.95      0.92       141

    accuracy                           0.92       783
   macro avg       0.92      0.92      0.92       783
weighted avg       0.92      0.92      0.92       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [29]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

print("Confusion Matrix for Test Set 2:\n", confusion_matrix(y2, grid.predict(X2)))
print("Classification Report for Test Set 2:\n", classification_report(y2, grid.predict(X2)))

Confusion Matrix for Test Set 2:
 [[  0   0  16]
 [  0   0 206]
 [  0   0  75]]
Classification Report for Test Set 2:
               precision    recall  f1-score   support

        rest       0.00      0.00      0.00        16
    straight       0.00      0.00      0.00       206
        turn       0.25      1.00      0.40        75

    accuracy                           0.25       297
   macro avg       0.08      0.33      0.13       297
weighted avg       0.06      0.25      0.10       297



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


___

#### 1.3.3 - Questions

**Based on results so far**
1. Comment on the model's performances on the datasets *(X_test, y_test)*, *(X1, y1)*, *(X2, y2)*? A significant difference in performance should be observed, what is this an example of? (3 marks)
1. Provide and explain potential causes for the observed behaviour in performance? (3 marks)

#### 1.3.3 - Answers
1.
The model performs well on the (X_test, y_test) dataset, achieving high precision, recall, and F1-scores across all three classes, showing strong predictive accuracy. On (X1, y1), the performance is slightly lower, with a notable drop for the "rest" class, although it still demonstrates good overall accuracy. However, the model performs poorly on (X2, y2), with very low precision, recall, and F1-scores for the "rest" and "straight" classes, failing to correctly classify instances from these categories. This significant difference in performance across datasets, particularly the poor results on (X2, y2), highlights overfitting. The model appears to have learned patterns specific to the training data but struggles to generalize to unseen or distributionally different datasets like (X2, y2).

2.
The observed drop in performance on (X2, y2) may stem from the following specific causes:

Data Distribution Differences: (X2, y2) may differ significantly in feature or class distributions compared to the training or validation data, making it harder for the model to generalize.

Class Imbalance: If the "rest" and "straight" classes are underrepresented in the training data, the model might not have learned these categories well, leading to poor performance on (X2, y2).

Insufficient Training Data: A smaller or less diverse training dataset may have limited the model’s ability to capture broader patterns needed for generalization.

Noise in (X2, y2): Errors or inconsistencies in the labels or features of (X2, y2) could degrade performance specifically for this dataset.

___

## Section 2 - Classification Using *Selected Features* Derived From Inertial Sensor Data (12 Marks)

In this second section, an *SVC* model will be trained and tuned using grid search, however, only *selected features* will be used.

___

### 2.1 Grid Search with Cross Validation (5 Marks)

___

#### 2.1.1 Extract *selected* features and target from *cv_set*

From the *cv_set* dataframe, extract the selected features, listed below, and load them in a new dataframe *X*.
* *ax_var* - variance of accelerometer readings in x
* *ay_var* - variance of accelerometer readings in y
* *az_sum* - sum of accelerometer z readings
* *a_sum* - sum of the norm of accelerometer x, y, z readings
* *gz_var* - variance of gyroscope reading in z

In [30]:
# TODO - Load selected features from cv_set to X
# (1 marks)
#X =
selected_features = ["ax_var", "ay_var", "az_sum", "a_sum", "gz_var"]
X = cv_set[selected_features]
X.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,8.046094e-07,1.319665e-07,9.789349,9.797418,7.446916e-10
1,1.032619e-06,1.073067e-07,9.789095,9.798815,6.983481e-10


In [31]:
# TODO - Load motion_state column (target vector) into y
# y =
y = cv_set["motion_state"]

___

#### 2.1.2 - Configure and Perform Grid Search with Cross-Validation

Follow the steps from **1.2.2** and perform model training and tuning using the selected feature set.

In [32]:
# TODO - Perform model training and tuning using `GridSearchCV`
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
pipe = Pipeline([("scaler", StandardScaler()), ("svc", SVC())])
param_grid = {
    "scaler": [StandardScaler(), RobustScaler(), None],
    "svc__C": [0.01, 0.1, 1, 10, 100],
    "svc__gamma": [0.001, 0.01, 0.1, 1],
    "svc__kernel": ["rbf"],
}
grid = GridSearchCV(pipe, param_grid, cv=5, scoring="accuracy", return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'scaler': [StandardScaler(), RobustScaler(), None],
                         'svc__C': [0.01, 0.1, 1, 10, 100],
                         'svc__gamma': [0.001, 0.01, 0.1, 1],
                         'svc__kernel': ['rbf']},
             return_train_score=True, scoring='accuracy')

___

#### 2.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [33]:
# TODO: Print the results from the grid search

#print("Best params:\n{}\n".format(???))
#print("Best cross-validation train score: {:.2f}".format(???))
#print("Best cross-validation validation score: {:.2f}".format(???))
#print("Test-set score: {:.2f}".format(???)))

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format(grid.cv_results_['mean_train_score'][grid.best_index_]))
print("Best cross-validation validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'scaler': RobustScaler(), 'svc__C': 100, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}

Best cross-validation train score: 0.84
Best cross-validation validation score: 0.82
Test-set score: 0.83


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [34]:
# TODO - Display confusion matrix and print classification reports
print("Confusion Matrix:\n", confusion_matrix(y_test, grid.predict(X_test)))
print("Classification Report:\n", classification_report(y_test, grid.predict(X_test)))

Confusion Matrix:
 [[154   1   0]
 [ 20 173  14]
 [  1  44  54]]
Classification Report:
               precision    recall  f1-score   support

        rest       0.88      0.99      0.93       155
    straight       0.79      0.84      0.81       207
        turn       0.79      0.55      0.65        99

    accuracy                           0.83       461
   macro avg       0.82      0.79      0.80       461
weighted avg       0.82      0.83      0.82       461



___

#### 2.1.4 - Question


1. Consider the differences between the performance on the training/cross-validation dataset for section 1 (1.2.3) and section 2 (2.1.3). Which model is a better fit based on *just these two evaluations*? (4 marks)

#### 2.1.4 Answer
Based on the results, the model from Section 1 achieves a higher performance across all metrics compared to the model from Section 2. Specifically, Section 1's best cross-validation train score is 0.98, its validation score is 0.95, and its test-set score is 0.96. In contrast, Section 2's model has a best cross-validation train score of 0.84, a validation score of 0.82, and a test-set score of 0.83. This indicates that the Section 1 model, trained on all features, captures more information and performs better at classification tasks.

The Section 2 model, which uses a reduced set of selected features, shows a decline in performance, suggesting that the reduced feature set is less effective for classification. While this model might reduce overfitting (as seen in the lower training score), the drop in validation and test performance indicates that the selected features may not represent the data comprehensively.
Thus, the Section 1 model is a better fit for this task, given its consistently higher performance across training and testing datasets.

___

### 2.2 - Model Evaluation on Test Datasets 1 and 2 (7 Marks)

___

#### 2.2.1 - Extract selected features and targets from test datasets

The model trained in 2.1 is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into the selected features and target vectors *(X1, y1)* and (X2,)

In [35]:
# TODO - Load the target vector from test_set1 into y1, and the selected features from test_set1 into X1
# (1 marks)
# y1 =
# X1 =
y1 = test_set1["motion_state"]
X1 = test_set1[selected_features]
X1.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000001,7.749919e-08,9.786930,9.801661,7.727190e-10
1,0.000001,9.243000e-08,9.788127,9.802481,8.871433e-10


In [36]:
# TODO - Load the target vector from test_set2 into y2, and the selected features from test_set2 into X2
# y2 =
# X2 =

y2 = test_set2["motion_state"]
X2 = test_set2[selected_features]
X2.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000002,3.020959e-07,9.791451,9.800683,1.405038e-09
1,0.000003,6.859937e-07,9.785619,9.797741,1.164381e-09


___

#### 2.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [37]:
# TODO - Print test score accuracies for test_set1 and test_set2
#print("test_set1 score: {:.2f}".format(???))
#print("test_set2 score: {:.2f}".format(???))

print("Test Set 1 Accuracy: {:.2f}".format(grid.score(X1, y1)))
print("Test Set 2 Accuracy: {:.2f}".format(grid.score(X2, y2)))

Test Set 1 Accuracy: 0.83
Test Set 2 Accuracy: 0.84


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [38]:
# TODO - Display confusion matrix and print classification report for (X1, y1)

print("Confusion Matrix for Test Set 1:\n", confusion_matrix(y1, grid.predict(X1)))
print("Classification Report for Test Set 1:\n", classification_report(y1, grid.predict(X1)))

Confusion Matrix for Test Set 1:
 [[229   2   0]
 [ 26 336  49]
 [  0  57  84]]
Classification Report for Test Set 1:
               precision    recall  f1-score   support

        rest       0.90      0.99      0.94       231
    straight       0.85      0.82      0.83       411
        turn       0.63      0.60      0.61       141

    accuracy                           0.83       783
   macro avg       0.79      0.80      0.80       783
weighted avg       0.83      0.83      0.83       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [39]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

print("Confusion Matrix for Test Set 2:\n", confusion_matrix(y2, grid.predict(X2)))
print("Classification Report for Test Set 2:\n", classification_report(y2, grid.predict(X2)))

Confusion Matrix for Test Set 2:
 [[ 15   1   0]
 [  0 178  28]
 [  0  18  57]]
Classification Report for Test Set 2:
               precision    recall  f1-score   support

        rest       1.00      0.94      0.97        16
    straight       0.90      0.86      0.88       206
        turn       0.67      0.76      0.71        75

    accuracy                           0.84       297
   macro avg       0.86      0.85      0.85       297
weighted avg       0.85      0.84      0.84       297



___

#### 2.2.3 Questions

1. For section 2, compare between the model's performance on the training/validation (2.1) set to the the performance on the two testing sets (2.2). How does the model fit? (3 marks)
1. Compare the models' performances and fits between section 1 and section 2, and consider all three datasets. (3 marks)

#### 2.2.3 Answers
**1.**
The model in Section 2 demonstrates consistent performance across the training/validation dataset and the two independent test sets, with minor variations. On the training/validation set (2.1), the model achieves a cross-validation train score of 0.84, a validation score of 0.82, and a test-set accuracy of 0.83. According to the confusion matrix and classification report, the model performs well for the "rest" and "straight" classes but encounters challenges with the "turn" class, where recall is notably lower at 0.55.

For Test Set 1, the accuracy remains at 0.83, with strong performance on "rest" and "straight" classes, but the "turn" class continues to show weaker precision and recall (f1-score of 0.61). On Test Set 2, the model's accuracy slightly increases to 0.84, excelling in the "rest" and "straight" categories, while the "turn" class shows moderate improvement with a recall of 0.76 and precision of 0.67. Overall, the model is well-suited for the data but could benefit from further refinement to better handle the "turn" class, which remains the most challenging to classify.


**2.**
In Section 1, the SVC model using all features achieves high scores on the training/validation dataset, with a train score of 0.98, validation score of 0.95, and test-set score of 0.96. However, the model struggles to generalize, as seen in its poor performance on Test Set 2, with an accuracy of only 0.25. While it performs well on Test Set 1 (0.92), the sharp drop on Test Set 2 indicates overfitting, particularly evident in the "rest" and "straight" classes, where precision and recall drop to 0.00.

In contrast, Section 2 focuses on selected features, resulting in more consistent performance. The model achieves a train score of 0.84, validation score of 0.82, and test-set score of 0.83. It also performs comparably on both Test Set 1 (0.83) and Test Set 2 (0.84), showing improved generalization. Although it still struggles with some classes, it avoids the overfitting observed in Section 1 and maintains a better balance between training and testing performance across datasets.


____

## Section 3 - Classification Using *Group Selected Features* From Inertial Sensor Data (10 Marks)

Once again, an *SVC* model is to be trained and tuned using grid search. However, you must select which features to use from the ones provided in *cv_set*.  

____

### 3.1 Grid Search with Cross Validation (4 Marks)

___

#### 3.1.1 Extract *Group Selected* features and target from *cv_set*

**Question**
1. Please list the features that you have selected and explain *why you have chosen* these features? (4 marks)


**Answer**

Selected Features: The chosen features are derived from accelerometer and gyroscope data. Specifically, a_sum (sum of accelerometer readings) and g_sum (sum of gyroscope readings) represent the overall motion magnitude and rotational activity, providing insights into total movement dynamics. Additionally, a_min (minimum accelerometer reading) and g_min (minimum gyroscope reading) capture the lowest motion and rotational activity, useful for identifying periods of rest or inactivity. Finally, a_max (maximum accelerometer reading) and g_max (maximum gyroscope reading) highlight peak accelerations and rotations, essential for identifying high-activity events.

Explanation: These features leverage both accelerometer and gyroscope data to offer a comprehensive view of motion and rotational behavior. The selected features are particularly valuable for detecting and classifying different activity states, balancing information about total movement (sum), extremes (minimum and maximum), and variations in motion dynamics. While they may be sensitive to noise and bias, their interpretability and utility in summarizing activity across time windows make them ideal for classification tasks.

In [40]:
# TODO - Load group selected features from cv_set to X
# X =
selected_group_features = ["a_sum", "g_sum","a_min", "g_min","a_max", "g_max"]
X = cv_set[selected_group_features]
X.head(2)

,a_sum,g_sum,a_min,g_min,a_max,g_max
0,9.797418,0.022554,0.023871,0.000003,0.024997,0.000187
1,9.798815,0.021731,0.024153,0.000006,0.024829,0.000144


In [41]:
# TODO - Load motion_state column (target vector) into y
# y =
y = cv_set["motion_state"]

___

#### 3.1.2 - Configure and Perform Grid Search with Cross-Validation

Follow the steps from **1.2.2** and perform model training and tuning using the group selected feature set.

In [42]:
# TODO - Perform model training and tuning using `GridSearchCV`

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Define pipeline
pipe = Pipeline([("scaler", StandardScaler()), ("svc", SVC())])

# Define parameter grid for GridSearchCV
param_grid = {
    "scaler": [StandardScaler(), RobustScaler(), None],
    "svc__C": [0.01, 0.1, 1, 10, 100],
    "svc__gamma": [0.001, 0.01, 0.1, 1],
    "svc__kernel": ["rbf"],
}

# Perform Grid Search
grid = GridSearchCV(pipe, param_grid, cv=5, scoring="accuracy", return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'scaler': [StandardScaler(), RobustScaler(), None],
                         'svc__C': [0.01, 0.1, 1, 10, 100],
                         'svc__gamma': [0.001, 0.01, 0.1, 1],
                         'svc__kernel': ['rbf']},
             return_train_score=True, scoring='accuracy')

___

#### 3.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [43]:
# TODO: Print the results from the grid search

#print("Best params:\n{}\n".format(???))
#print("Best cross-validation train score: {:.2f}".format(???))
#print("Best cross-validation validation score: {:.2f}".format(???))
#print("Test-set score: {:.2f}".format(???)))

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format((grid.cv_results_['mean_train_score'][grid.best_index_])))
print("Best cross-validation validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'scaler': RobustScaler(), 'svc__C': 100, 'svc__gamma': 0.1, 'svc__kernel': 'rbf'}

Best cross-validation train score: 0.96
Best cross-validation validation score: 0.94
Test-set score: 0.95


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [44]:
# TODO - Display confusion matrix and print classification report

print("Confusion Matrix:\n", confusion_matrix(y_test, grid.predict(X_test)))
print("Classification Report:\n", classification_report(y_test, grid.predict(X_test)))

Confusion Matrix:
 [[153   2   0]
 [ 11 194   2]
 [  0   9  90]]
Classification Report:
               precision    recall  f1-score   support

        rest       0.93      0.99      0.96       155
    straight       0.95      0.94      0.94       207
        turn       0.98      0.91      0.94        99

    accuracy                           0.95       461
   macro avg       0.95      0.94      0.95       461
weighted avg       0.95      0.95      0.95       461



___

### 3.2 - Model Evaluation on Test Datasets 1 and 2 (6 Marks)

___

#### 3.2.1 - Extract group selected features and targets from test datasets

The model trained in 3.1 is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into group selected features and target vectors *(X1, y1)* and *(X2,y2)*

In [45]:
# TODO - Load the target vector from test_set1 into y1, and the group selected features from test_set1 into X1
# y1 =
# X1 =

y1 = test_set1["motion_state"]
X1 = test_set1[selected_group_features]

X1.head(2)

,a_sum,g_sum,a_min,g_min,a_max,g_max
0,9.801661,0.025634,0.024056,0.000003,0.024886,0.000170
1,9.802481,0.025218,0.024084,0.000007,0.024883,0.000201


In [46]:
# TODO - Load the target vector from test_set2 into y2, and the group selected features from test_set2 into X2
# y2 =
# X2 =

y2 = test_set2["motion_state"]
X2 = test_set2[selected_group_features]

X2.head(2)

,a_sum,g_sum,a_min,g_min,a_max,g_max
0,9.800683,0.012437,0.048406,0.000007,0.049764,0.000142
1,9.797741,0.013805,0.047696,0.000013,0.050763,0.000151


___

#### 3.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [47]:
# TODO - Print test score accuracies for test_set1 and test_set2
#print("test_set1 score: {:.2f}".format(???))
#print("test_set2 score: {:.2f}".format(???))

print("Test Set 1 Accuracy: {:.2f}".format(grid.score(X1, y1)))
print("Test Set 2 Accuracy: {:.2f}".format(grid.score(X2, y2)))

Test Set 1 Accuracy: 0.78
Test Set 2 Accuracy: 0.25


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [48]:
# TODO - Display confusion matrix and print classification report for (X1, y1)

print("Confusion Matrix for Test Set 1:\n", confusion_matrix(y1, grid.predict(X1)))
print("Classification Report for Test Set 1:\n", classification_report(y1, grid.predict(X1)))

Confusion Matrix for Test Set 1:
 [[104 127   0]
 [  3 392  16]
 [  0  23 118]]
Classification Report for Test Set 1:
               precision    recall  f1-score   support

        rest       0.97      0.45      0.62       231
    straight       0.72      0.95      0.82       411
        turn       0.88      0.84      0.86       141

    accuracy                           0.78       783
   macro avg       0.86      0.75      0.77       783
weighted avg       0.82      0.78      0.77       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [49]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

print("Confusion Matrix for Test Set 2:\n", confusion_matrix(y2, grid.predict(X2)))
print("Classification Report for Test Set 2:\n", classification_report(y2, grid.predict(X2)))

Confusion Matrix for Test Set 2:
 [[  0   0  16]
 [  0   0 206]
 [  0   0  75]]
Classification Report for Test Set 2:
               precision    recall  f1-score   support

        rest       0.00      0.00      0.00        16
    straight       0.00      0.00      0.00       206
        turn       0.25      1.00      0.40        75

    accuracy                           0.25       297
   macro avg       0.08      0.33      0.13       297
weighted avg       0.06      0.25      0.10       297



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


___

#### 3.2.3 Questions

1. Based on the observations across all three datasets, compare the performance of the model trained in this section to: (3 marks)
    1. Model trained in Section 1 using *all features*.
    1. Model trained in Section 2 using *selected features*.

2. Please proivde potential explanations for the differences observed in models' performances across the three sections. (3 marks)

#### 3.2.3 Answers
**1.** Comparison of Models Across Sections 1, 2, and 3:

The Section 1 model, which uses all features, demonstrates strong performance on the training, validation, and primary test dataset, achieving a train score of 0.98, validation score of 0.95, and test accuracy of 0.96. However, its performance drops significantly on Test Set 2, with an accuracy of only 0.25, suggesting overfitting or poor adaptability to domain shifts in this dataset.

The Section 2 model, trained with a reduced set of selected features, achieves more balanced results with a train score of 0.84, validation score of 0.82, and test accuracy of 0.83. It also performs consistently across both test datasets, with Test Set 2 accuracy reaching 0.84, highlighting its robustness and ability to generalize to unseen data better. Meanwhile, the Section 3 model, using grouped features, achieves high scores during training (train score of 0.96, validation score of 0.94, test score of 0.95) but struggles on Test Set 1 (0.78) and fails entirely on Test Set 2 (0.25), similar to the Section 1 model.


**2.** Reasons for Observed Differences:

Feature Selection and Model Complexity:
Section 1’s model uses all features, leading to higher complexity and overfitting, which results in poor adaptability to datasets with domain differences (Test Set 2). Section 2’s focus on selected, meaningful features reduces noise and enhances generalization, improving performance across diverse datasets. In Section 3, grouped features improve performance during training but fail to generalize effectively due to noise or dataset-specific biases.

Dataset Variability and Domain Shift:
Test Set 2 likely represents a domain shift with significantly different data distributions. The Section 1 and Section 3 models, which rely on a larger or grouped feature set, struggle to handle this variability, leading to poor results. Section 2's smaller and more focused feature set makes it more robust to such shifts, resulting in better performance across all datasets.

____

## Section 4 - Classification Using Principal Components of Features From Inertial Sensor Data (10 Marks)

____

### 4.1 Principal Component Analysis (PCA) Search with Cross Validation (4 Marks)

___

#### 4.1.1 Extract *ALL* features and target from *cv_set*

In [78]:
# TODO - Load all features from cv_set to X
# X =

X = cv_set.drop(columns=["motion_state"])
X.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.003929,-0.093081,9.789349,-0.002649,0.001406,-0.001391,9.797418,0.022554,8.046094e-07,1.319665e-07,...,0.023871,0.000003,0.001496,0.000544,0.024992,0.000110,0.000075,0.000078,0.024997,0.000187
1,-0.000966,-0.092417,9.789095,-0.001806,0.000103,-0.001943,9.798815,0.021731,1.032619e-06,1.073067e-07,...,0.024153,0.000006,0.001865,0.000615,0.024828,0.000124,0.000098,0.000090,0.024829,0.000144


In [79]:
# TODO - Load motion_state column (target vector) into y
# y =

y = cv_set["motion_state"]

___

#### 4.1.2 - Configure and Perform Grid Search with Cross-Validation

Similar to the steps from **1.2.2** and perform model training and tuning for an `SVC` model. Perform the `train_test_split`, initiate a `Pipeline`, then define the *Parameters* for `GridSearchCV`. Finally, perfrom the grid search.

In [80]:
# TODO - Perform model training and tuning using `GridSearchCV`
# X_train, X_test, y_train, y_test =

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In the `Pipeline` add an entry *'pca'* to apply `PCA`.

In [81]:
# Perform any missing imports

# TODO - Initiate the pipeline
# (1 marks)
# pipe =

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA()),
    ("svc", SVC())
])

Similar to previous sections, define the grid search configuration (*param_grid*). Add *n_components* hyper parameter tuning for PCA, and search over the values `n_components=[4,7,11,14,16]`.  

In [82]:
# TODO - Define the parameter grid for your model
# (3 marks)
# param_grid =

param_grid = {
    "pca__n_components": [4, 7, 11, 14, 16],
    "svc__C": [0.01, 0.1, 1, 10, 100],
    "svc__gamma": [0.001, 0.01, 0.1, 1],
    "svc__kernel": ["rbf"]
}

In [83]:
# Perform the grid search to train model and tune hyper parameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()), ('svc', SVC())]),
             param_grid={'pca__n_components': [4, 7, 11, 14, 16],
                         'svc__C': [0.01, 0.1, 1, 10, 100],
                         'svc__gamma': [0.001, 0.01, 0.1, 1],
                         'svc__kernel': ['rbf']},
             return_train_score=True, scoring='accuracy')

___

#### 4.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [84]:
# TODO: Print the results from the grid search

#print("Best params:\n{}\n".format(???))
#print("Best cross-validation train score: {:.2f}".format(???))
#print("Best cross-validation validation score: {:.2f}".format(???))
#print("Test-set score: {:.2f}".format(???)))

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format((grid.cv_results_['mean_train_score'][grid.best_index_])))
print("Best cross-validation validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'pca__n_components': 16, 'svc__C': 100, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}

Best cross-validation train score: 0.97
Best cross-validation validation score: 0.94
Test-set score: 0.94


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [85]:
# TODO - Display confusion matrix and print classification report

print("Confusion Matrix:\n", confusion_matrix(y_test, grid.predict(X_test)))
print("Classification Report:\n", classification_report(y_test, grid.predict(X_test)))

Confusion Matrix:
 [[151   4   0]
 [ 11 195   1]
 [  0  10  89]]
Classification Report:
               precision    recall  f1-score   support

        rest       0.93      0.97      0.95       155
    straight       0.93      0.94      0.94       207
        turn       0.99      0.90      0.94        99

    accuracy                           0.94       461
   macro avg       0.95      0.94      0.94       461
weighted avg       0.94      0.94      0.94       461



___

### 4.2 - Model Evaluation on Test Datasets 1 and 2 (6 Marks)

___

#### 4.2.1 - Extract group selected features and targets from test datasets

In [87]:
# TODO - Load the target vector from test_set1 into y1, and all features from test_set1 into X1
# y1 =
# X1 =

y1 = test_set1["motion_state"]
X1 = test_set1.drop(columns=["motion_state"])
X1.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,0.264465,0.089904,9.786930,-0.002305,0.000218,-0.001143,9.801661,0.025634,0.000001,7.749919e-08,...,0.024056,0.000003,0.002506,0.000845,0.024884,0.000155,0.000078,0.000069,0.024886,0.000170
1,0.264300,0.088882,9.788127,-0.001865,-0.000523,-0.001999,9.802481,0.025218,0.000001,9.243000e-08,...,0.024084,0.000007,0.002427,0.000949,0.024881,0.000201,0.000095,0.000076,0.024883,0.000201


In [88]:
# TODO - Load the target vector from test_set2 into y2, and all features from test_set2 into X2
# y2 =
# X2 =

y2 = test_set2["motion_state"]
X2 = test_set2.drop(columns=["motion_state"])
X2.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.046400,0.305907,9.791451,-0.002184,0.001311,-0.001901,9.800683,0.012437,0.000002,3.020959e-07,...,0.048406,0.000007,0.002575,0.002993,0.049701,0.000094,0.000115,0.000080,0.049764,0.000142
1,-0.050073,0.311947,9.785619,-0.003369,0.001898,-0.002051,9.797741,0.013805,0.000003,6.859937e-07,...,0.047696,0.000013,0.003880,0.004106,0.050477,0.000140,0.000140,0.000083,0.050763,0.000151


___

#### 4.2.2 - Visualize Results

In [89]:
# TODO - Print test score accuracies for test_set1 and test_set2
#print("test_set1 score: {:.2f}".format(???))
#print("test_set2 score: {:.2f}".format(???))

print("Test Set 1 Accuracy: {:.2f}".format(grid.score(X1, y1)))
print("Test Set 2 Accuracy: {:.2f}".format(grid.score(X2, y2)))

Test Set 1 Accuracy: 0.91
Test Set 2 Accuracy: 0.40


In [90]:
# TODO - Display confusion matrix and print classification report for (X1, y1)

print("Confusion Matrix for Test Set 1:\n", confusion_matrix(y1, grid.predict(X1)))
print("Classification Report for Test Set 1:\n", classification_report(y1, grid.predict(X1)))

Confusion Matrix for Test Set 1:
 [[227   4   0]
 [ 16 384  11]
 [  0  42  99]]
Classification Report for Test Set 1:
               precision    recall  f1-score   support

        rest       0.93      0.98      0.96       231
    straight       0.89      0.93      0.91       411
        turn       0.90      0.70      0.79       141

    accuracy                           0.91       783
   macro avg       0.91      0.87      0.89       783
weighted avg       0.91      0.91      0.90       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [91]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

print("Confusion Matrix for Test Set 2:\n", confusion_matrix(y2, grid.predict(X2)))
print("Classification Report for Test Set 2:\n", classification_report(y2, grid.predict(X2)))

Confusion Matrix for Test Set 2:
 [[  0  16   0]
 [  0  45 161]
 [  0   0  75]]
Classification Report for Test Set 2:
               precision    recall  f1-score   support

        rest       0.00      0.00      0.00        16
    straight       0.74      0.22      0.34       206
        turn       0.32      1.00      0.48        75

    accuracy                           0.40       297
   macro avg       0.35      0.41      0.27       297
weighted avg       0.59      0.40      0.36       297



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


___

#### 4.2.3 - Questions

1. Based on the observations across all three datasets, compare the performance of the model with PCA, trained in this section to: (4 marks)
    * Model trained in Section 1 using *all features*.
    * Model trained in Section 2 using *selected features*.
    * Model trained in Section 3 using *group selected features*

2. Please provide potential explanations for the differences observed in models' performances across between section 1 and 4. (2 marks)

#### 4.2.3 - Answers

____

## Section 5 - Classification Using *Pre Selected Features* Derived From Inertial Sensor Data (10 Marks)

___

Now, a second classifer is to be trained and tuned using grid search. The **selected features from section 2** must be used. You may choose from one of the algorithms discussed that class listed below.
1. Random Forest
1. K-Nearest Neighbors
1. Gradient Boosting Machines
1. Neural Networks

____

### 5.1 Grid Search with Cross Validation (6 Marks)

___

#### 5.1.1 Extract *Selected* features and target from *cv_set*

**Questions**
1. Please list the ML model that you have selected . And explain *why you have chosen* this model? (2 marks)

**Answer**
1.

In [92]:
# TODO - Load group selected features from cv_set to X
# X =

selected_features = ["ax_var", "ay_var", "az_sum", "a_sum", "gz_var"]
X = cv_set[selected_features]

X.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,8.046094e-07,1.319665e-07,9.789349,9.797418,7.446916e-10
1,1.032619e-06,1.073067e-07,9.789095,9.798815,6.983481e-10


In [93]:
# TODO - Load motion_state column (target vector) into y
# y =

y = cv_set["motion_state"]

___

#### 5.1.2 - Configure and Perform Grid Search with Cross-Validation

Similar to the steps from **1.2.2** and perform model training and tuning using for the model selected. Perform the `train_test_split`, initiate a `Pipeline`, then define the *Parameters* for `GridSearchCV`. Finally, perfrom the grid search.

In [94]:
# TODO - Perform train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [95]:
# Perform any missing imports

# TODO - Initiate the pipeline
# (1 mark)
# pipe =

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("rf", RandomForestClassifier(random_state=42))
])

In [96]:
# TODO - Define the parameter grid for your model
# (3 marks)
# param_grid =

param_grid = {
    "rf__n_estimators": [50, 100, 200],
    "rf__max_depth": [None, 10, 20],
    "rf__min_samples_split": [2, 5, 10],
}

In [97]:
# TODO - Perform the grid search
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('rf',
                                        RandomForestClassifier(random_state=42))]),
             param_grid={'rf__max_depth': [None, 10, 20],
                         'rf__min_samples_split': [2, 5, 10],
                         'rf__n_estimators': [50, 100, 200]},
             return_train_score=True, scoring='accuracy')

___

#### 5.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [98]:
# TODO: Print the results from the grid search

#print("Best params:\n{}\n".format(???))
#print("Best cross-validation train score: {:.2f}".format(???))
#print("Best cross-validation validation score: {:.2f}".format(???))
#print("Test-set score: {:.2f}".format(???)))

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format(max(grid.cv_results_['mean_train_score'])))
print("Best cross-validation validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'rf__max_depth': 10, 'rf__min_samples_split': 2, 'rf__n_estimators': 200}

Best cross-validation train score: 1.00
Best cross-validation validation score: 0.85
Test-set score: 0.84


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [99]:
# TODO - Display confusion matrix and print classification report
print("Confusion Matrix:\n", confusion_matrix(y_test, grid.predict(X_test)))
print("Classification Report:\n", classification_report(y_test, grid.predict(X_test)))

Confusion Matrix:
 [[147   8   0]
 [  5 181  21]
 [  1  39  59]]
Classification Report:
               precision    recall  f1-score   support

        rest       0.96      0.95      0.95       155
    straight       0.79      0.87      0.83       207
        turn       0.74      0.60      0.66        99

    accuracy                           0.84       461
   macro avg       0.83      0.81      0.82       461
weighted avg       0.84      0.84      0.84       461



___

### 5.2 - Model Evaluation on Test Datasets 1 and 2 (4 Marks)

___

#### 5.2.1 - Extract selected features and targets from test datasets

The model trained in 5.1, is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into features and target vectors *(X1, y1)* and *(X2,y2)*

In [100]:
# TODO - Load the target vector from test_set1 into y1, and the selected features from test_set1 into X1
# y1 =
# X1 =

y1 = test_set1["motion_state"]
X1 = test_set1[selected_features]

X1.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000001,7.749919e-08,9.786930,9.801661,7.727190e-10
1,0.000001,9.243000e-08,9.788127,9.802481,8.871433e-10


In [101]:
# TODO - Load the target vector from test_set2 into y2, and the selected features from test_set2 into X2
# y2 =
# X2 =

y2 = test_set2["motion_state"]
X2 = test_set2[selected_features]

X2.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000002,3.020959e-07,9.791451,9.800683,1.405038e-09
1,0.000003,6.859937e-07,9.785619,9.797741,1.164381e-09


___

#### 5.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [102]:
# TODO - Print test score accuracies for test_set1 and test_set2
#print("test_set1 score: {:.2f}".format(???))
#print("test_set2 score: {:.2f}".format(???))

print("Test Set 1 Accuracy: {:.2f}".format(grid.score(X1, y1)))
print("Test Set 2 Accuracy: {:.2f}".format(grid.score(X2, y2)))

Test Set 1 Accuracy: 0.84
Test Set 2 Accuracy: 0.70


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [103]:
# TODO - Display confusion matrix and print classification report for (X1, y1)

print("Confusion Matrix for Test Set 1:\n", confusion_matrix(y1, grid.predict(X1)))
print("Classification Report for Test Set 1:\n", classification_report(y1, grid.predict(X1)))

Confusion Matrix for Test Set 1:
 [[227   4   0]
 [  3 354  54]
 [  0  62  79]]
Classification Report for Test Set 1:
               precision    recall  f1-score   support

        rest       0.99      0.98      0.98       231
    straight       0.84      0.86      0.85       411
        turn       0.59      0.56      0.58       141

    accuracy                           0.84       783
   macro avg       0.81      0.80      0.80       783
weighted avg       0.84      0.84      0.84       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [104]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

print("Confusion Matrix for Test Set 2:\n", confusion_matrix(y2, grid.predict(X2)))
print("Classification Report for Test Set 2:\n", classification_report(y2, grid.predict(X2)))

Confusion Matrix for Test Set 2:
 [[ 11   5   0]
 [  0 126  80]
 [  0   4  71]]
Classification Report for Test Set 2:
               precision    recall  f1-score   support

        rest       1.00      0.69      0.81        16
    straight       0.93      0.61      0.74       206
        turn       0.47      0.95      0.63        75

    accuracy                           0.70       297
   macro avg       0.80      0.75      0.73       297
weighted avg       0.82      0.70      0.72       297



___

#### 5.2.3 Question

1. For the model trainied in this section, compare its performance to the performance of SVC which was also trained with the same selected features (section 2). Which model fits better? Explain causes for any observed discrepancies? (4 marks)

#### 5.2.3 - Answer
1.


____